In [1]:
import numpy as np
from numpy import cos
from numpy import sin
from math import radians
import math as mt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Activation, Dropout, Input
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
radians(360)

6.283185307179586

In [3]:
l1 = 50
l2 = 40
l3 = 20
Px=[]
Py=[]
phi=[]
tt1=[]
tt2=[]
tt3=[]
for i in range(-90,90,1):
  for j in range(-90,90,1):
    for k in range(-90,90,1):
      Px.append(l1*cos(radians(i))+l2*cos(radians(i+j))+radians(k)*cos(radians(i+j+k)))
      Py.append(l1*sin(radians(i))+l2*sin(radians(i+j))+l3*sin(radians(i+j+k)))
      phi.append(radians(i+j+k))
      tt1.append(radians(i))
      tt2.append(radians(j))
      tt3.append(radians(k))

In [4]:
data = pd.DataFrame()
data["Px"]=Px
data["Py"]=Py
data["phi"]=phi
data["tt1"]=tt1
data["tt2"]=tt2
data["tt3"]=tt3

In [5]:
data

,Px,Py,phi,tt1,tt2,tt3
0,-40.000000,-30.000000,-4.712389,-1.570796,-1.570796,-1.570796
1,-39.972890,-30.003046,-4.694936,-1.570796,-1.570796,-1.553343
2,-39.946398,-30.012183,-4.677482,-1.570796,-1.570796,-1.535890
3,-39.920531,-30.027409,-4.660029,-1.570796,-1.570796,-1.518436
4,-39.895297,-30.048719,-4.642576,-1.570796,-1.570796,-1.500983
...,...,...,...,...,...,...
5831995,-39.283810,31.537442,4.590216,1.553343,1.553343,1.483530
5831996,-39.259908,31.497927,4.607669,1.553343,1.553343,1.500983
5831997,-39.235353,31.464471,4.625123,1.553343,1.553343,1.518436
5831998,-39.210151,31.437084,4.642576,1.553343,1.553343,1.535890


In [6]:
x = data.drop(['tt1','tt2','tt3'],axis=1)
x

,Px,Py,phi
0,-40.000000,-30.000000,-4.712389
1,-39.972890,-30.003046,-4.694936
2,-39.946398,-30.012183,-4.677482
3,-39.920531,-30.027409,-4.660029
4,-39.895297,-30.048719,-4.642576
...,...,...,...
5831995,-39.283810,31.537442,4.590216
5831996,-39.259908,31.497927,4.607669
5831997,-39.235353,31.464471,4.625123
5831998,-39.210151,31.437084,4.642576


In [7]:
y = data.drop(['Px','Py','phi'],axis=1)
y

,tt1,tt2,tt3
0,-1.570796,-1.570796,-1.570796
1,-1.570796,-1.570796,-1.553343
2,-1.570796,-1.570796,-1.535890
3,-1.570796,-1.570796,-1.518436
4,-1.570796,-1.570796,-1.500983
...,...,...,...
5831995,1.553343,1.553343,1.483530
5831996,1.553343,1.553343,1.500983
5831997,1.553343,1.553343,1.518436
5831998,1.553343,1.553343,1.535890


In [8]:
col_names = list(x.columns)
scaler=StandardScaler()
X_df = scaler.fit_transform(x)
X_df = pd.DataFrame(X_df, columns=col_names)
X_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Px,5832000.0,-8.471386e-15,1.0,-2.780380,-0.612863,2.644296e-01,0.792534,1.334330
Py,5832000.0,-1.158209e-14,1.0,-1.640894,-0.954866,-9.560669e-03,0.956959,1.661670
phi,5832000.0,1.249214e-17,1.0,-2.983379,-0.705566,1.474515e-16,0.705566,2.983379


In [9]:
x_train,x_test,y_train,y_test = train_test_split(X_df,y,test_size=0.2 , random_state=7)

In [10]:
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(3,)))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.summary()
model.compile(loss='mse',optimizer=RMSprop(),metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2048      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 3)                 771       
                                                                 
Total params: 265,731
Trainable params: 265,731
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(x_train,y_train,batch_size = 256,epochs=10,verbose=1,callbacks=[EarlyStopping(monitor='val_loss',patience=50)],validation_data = (x_test, y_test)) #,validation_split=0.2

Epoch 1/10
18225/18225 [==============================] - 252s 14ms/step - loss: 0.6875 - accuracy: 0.7822 - val_loss: 0.6842 - val_accuracy: 0.7914
Epoch 2/10
18225/18225 [==============================] - 251s 14ms/step - loss: 0.6846 - accuracy: 0.7933 - val_loss: 0.6828 - val_accuracy: 0.7995
Epoch 3/10
18225/18225 [==============================] - 248s 14ms/step - loss: 0.6841 - accuracy: 0.7954 - val_loss: 0.6836 - val_accuracy: 0.7929
Epoch 4/10
18225/18225 [==============================] - 248s 14ms/step - loss: 0.6838 - accuracy: 0.7969 - val_loss: 0.6829 - val_accuracy: 0.7967
Epoch 5/10
18225/18225 [==============================] - 249s 14ms/step - loss: 0.6835 - accuracy: 0.7975 - val_loss: 0.6820 - val_accuracy: 0.7954
Epoch 6/10
18225/18225 [==============================] - 244s 13ms/step - loss: 0.6838 - accuracy: 0.7970 - val_loss: 0.6840 - val_accuracy: 0.8018
Epoch 7/10
18225/18225 [==============================] - 247s 14ms/step - loss: 0.6839 - accuracy: 0.7961

In [12]:
model.evaluate (x_train, y_train)
score=model.evaluate(x_test,y_test,verbose=0)
print('sai so kiem tra la:',score[0])
print('do chinh xac kiem tra la:',score[1])

145800/145800 [==============================] - 344s 2ms/step - loss: 0.6831 - accuracy: 0.8039
sai so kiem tra la: 0.6829026937484741
do chinh xac kiem tra la: 0.8042001128196716


In [13]:
model.save('robot3.h5')

In [14]:
from tensorflow.keras.models import load_model
model1=load_model('robot3.h5')

In [15]:
y_test[4000:4001]

,tt1,tt2,tt3
3474861,0.296706,-0.802851,0.890118


In [16]:
result=model.predict(x_test[4000:4001])
result

array([[0.14756908, 0.15395415, 0.69847673]], dtype=float32)